In [1]:
import sys
sys.version

'3.7.4 (default, Oct 15 2019, 22:29:14) \n[GCC 7.4.0]'

http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/corpus/

Downloaded first part of the corpus with the `aws` command line interface, like

```aws s3 cp --no-sign-request s3://ai2-s2-research-public/open-corpus/2018-05-03/s2-corpus-00.gz .```

In [31]:
!ls semantic-scholar-raw/

license.txt   s2-corpus-00	    s2-corpus-clean.tmp
manifest.txt  s2-corpus-clean.json  s2-corpus-slim.json


In [32]:
!wc -l semantic-scholar-raw/s2-corpus-00

1000000 semantic-scholar-raw/s2-corpus-00


Filter out those lines that have empty entities, empty inCitations, and empty outCitations

In [23]:
!grep -v -e '"entities":\[\]' -e '"inCitations":\[\]' -e '"outCitations":\[\]' semantic-scholar-raw/s2-corpus-00 > semantic-scholar-raw/s2-corpus-clean.tmp
!wc -l semantic-scholar-raw/s2-corpus-clean.tmp

212901 semantic-scholar-raw/s2-corpus-clean.tmp


Filter out those rows with empty attributes

In [28]:
!grep -v -e '"journalName":""' -e '"title":""' -e '"journalVolume":""' -e '"id":""' -e '"ids":""' -e '"doi":""' -e '"doiUrl":""' semantic-scholar-raw/s2-corpus-clean.tmp > semantic-scholar-raw/s2-corpus-clean.json
!wc -l semantic-scholar-raw/s2-corpus-clean.json

86609 semantic-scholar-raw/s2-corpus-clean.json


Obtain a smaller sample from that data

In [29]:
!head -100 semantic-scholar-raw/s2-corpus-clean.json > semantic-scholar-raw/s2-corpus-slim.json

In [27]:
!head -1 semantic-scholar-raw/s2-corpus-slim.json | jq

{
  "entities": [
    "Eating",
    "Ethanol",
    "Isopropyl Alcohol 0.7 ML/ML Medicated Pad"
  ],
  "journalVolume": "31 4",
  "journalPages": "385-8",
  "pmid": "8879286v1",
  "year": 1996,
  "outCitations": [
    "40bb6636803d10626bfafa3dbf940fdac903d11c",
    "18ffb4ae9e0d8f0c43342c4c6b993f3ce34bd579",
    "827b793fbc923d3e068b2008dc02bb63e7069fb9",
    "b96cb6dd0bba1e9928eb6315e18339f667c6799f",
    "48431576480666445bdcb112533aa885d4ac1ad1",
    "1380d901dbce2dca8eba90501179485d4fe1e5d4",
    "4203ad4dc8530a92473e5576650eca84b51c9eee",
    "96b85a405b002e11199f967280ec9a9736543c86",
    "8d72d01b1b670c7bb65408e91d1705f0c95b1174",
    "7842615744a118a2d4a239787e9450c26c98c9f9",
    "6ed7439f9537fce3f82144843622b9c670a95d7e",
    "55787878e91f6efdf14e16438e733d80581f3bec",
    "e28468ccfa0ddce1d060d39d25305f6abab2eced",
    "5f6ba13f126408d25849c518bddf4812281ae970"
  ],
  "s2Url": "https://semanticscholar.org/paper/30aaf92bde9bf6cdc3dff237691fd5db55d4cb4a",
  "s2PdfUrl": "http://

## Parse JSON
https://markhneedham.com/blog/2015/07/23/neo4j-loading-json-documents-with-cypher/

In [8]:
import json
from py2neo import Graph
graph = Graph("localhost:7474")
SC_JSON = "semantic-scholar-raw/s2-corpus-000-slim"

In [29]:
# Send Cypher query.
graph.run("RETURN apoc.version();").data()

[{'apoc.version()': '3.5.0.9'}]

In [28]:
graph.run("call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;").data()

[{'name': 'Neo4j Kernel', 'version': '3.5.15', 'edition': 'enterprise'}]

In [9]:
query = """
WITH "myjson" AS url 
CALL apoc.load.json(url) 
YIELD value 
RETURN value 
LIMIT 1"""

graph.run(query).data()

[{'value': {'doiUrl': '',
   'venue': 'Alcohol and alcoholism',
   'journalName': 'Alcohol and alcoholism',
   'sources': ['Medline'],
   'year': 1996,
   'paperAbstract': 'The present study was designed to assess the temporal pattern of ethanol intake over a 24 h period in selectively bred, Sardinian alcohol-preferring (sP) rats. Ethanol intake occurred under the two-bottle, free choice regimen. sP rats consumed ethanol in three distinct peaks, rather regularly distributed over the 12 h dark phase of the light-dark cycle and positively correlated with food intake episodes. The temporal distribution of ethanol intake and estimated blood alcohol levels are consistent with the hypothesis that sP rats voluntarily drink ethanol for its pharmacological effects.',
   'inCitations': ['7f17780491b74f08b3b6fb603014835e74ecfdd3',
    '9e2664ede0d21990f25b7682f023bc11e79e89b9',
    '1dbf8f6f64447b6f311031a14a7b99e43fb51f44',
    '842707e424ae0e2beda89c8847ce7c0fafa60b92',
    'a0711d2594d6e72cab3

Here, `"myjson"` is an alias for the path to the file, as defined in https://neo4j.com/docs/labs/apoc/current/import/load-json/

## create Article instances

WITH "myjson" AS url 
CALL apoc.load.json(url) 
YIELD value as row
MERGE (a:Article {title: row.title, id: row.id, doi: row.doi})
RETURN a


```cypher
WITH "myjson" AS url 
CALL apoc.load.json(url) 
YIELD value as row
MERGE (art:Article {title: row.title, id: row.id, doi: row.doi, doi_url:row.doiUrl})
MERGE (jour:Journal {
	name: row.journalName,
    volume: row.journalVolume,
    pages: row.journalPages})-[:PUBLISHED_IN]->(art)
FOREACH (author in row.authors |
	MERGE (art)-[:AUTHORED_BY]->(auth:Author {name: author.name, id: author.ids})
)
FOREACH (k in row.entities |
	MERGE (art)-[:IS_ABOUT]->(keyw:Keyword {name: k})
)
FOREACH (f in row.fieldsOfStudy |
	MERGE (art)-[:IS_ABOUT]->(fos:FieldOfStudy {name: f})
)
MERGE (art)-[:IS_ABOUT]->(keyw)
```

- Adding conferences with venues and attaching them to articles
- Adding keywords
- Addind reviewers from authors and linking them to articles
- Adding incitations and outcitations

```cypher
WITH "myjson" AS url 
CALL apoc.load.json(url) 
YIELD value as row
UNWIND row.inCitations as cite_id
MATCH (src:Article {id: row.id}), (dest:Article {id: cite_id})
RETURN src, dest
```

## Flattening the CSV

In [22]:
import pandas as pd

with open(SC_JSON, encoding="UTF-8") as fp:
    data = [json.loads(line) for line in fp]

In [24]:
normalized_json_ = pd.json_normalize(data)
normalized_json.head()

,entities,journalVolume,journalPages,pmid,fieldsOfStudy,year,outCitations,s2Url,s2PdfUrl,id,authors,journalName,paperAbstract,inCitations,pdfUrls,title,doi,sources,doiUrl,venue
0,[],,,,[Geology],1984,[],https://semanticscholar.org/paper/2fc88e5e6980...,,2fc88e5e6980652109b40f15974891b55f2b9b53,"[{'name': 'David K. Elliott', 'ids': ['5036997...",,,[],[],"A new subfamily of the Pteraspididae (Agnatha,...",,[],,
1,[],66 1,\n 241-3\n,13395544,"[Medicine, Biology]",1957,[],https://semanticscholar.org/paper/9a7cc56f5190...,,9a7cc56f5190f56bbb13fac5134a7d44db754e96,"[{'name': 'Efraim Racker', 'ids': ['6799260']}...",Archives of biochemistry and biophysics,,[8c6cdbbeeaea83b9d10a310a1440d7b652337c99],[],Formation and utilization of octulose-8-phosph...,10.1016/0003-9861(57)90555-6,[Medline],https://doi.org/10.1016/0003-9861%2857%2990555-6,Archives of biochemistry and biophysics
2,[],,,,[Sociology],1983,[],https://semanticscholar.org/paper/4b677d50b18b...,,4b677d50b18be64018f6ae7ab5c88ae78baf02ad,"[{'name': 'Marguerita K. Summers', 'ids': ['28...",,A probe floating device including probe means ...,"[b5b58a261271e54006ee1464b5a345a553a8914f, 83c...",[],Teaching Heat--An Analysis of Misconceptions.,,[],,
3,[],,,,[Physics],2015,"[2497ed63572e8d5e5fe7945f0b23e0d090acd51c, 03b...",https://semanticscholar.org/paper/1b2f4e5be76a...,,1b2f4e5be76a0a746b72110b447b42fffa046b5c,"[{'name': 'Xiang Fa Liu', 'ids': ['153201706']...",,Abstract Experimental investigations on the ch...,[],[http://www.mechwork.ir/uploads/papers/4-Exper...,Experimental study on the characteristics of a...,10.1016/j.ijmultiphaseflow.2015.03.012,[],https://doi.org/10.1016/j.ijmultiphaseflow.201...,
4,[],,,,[Computer Science],2012,[],https://semanticscholar.org/paper/eb46626df0c8...,,eb46626df0c8eea987cb632b2354218073ee9cab,"[{'name': 'Zhang Ying', 'ids': ['144183558']}]",,Taking the Mars exploration trajectory design ...,[],[],The Gravity-assist Trajectory Design with Impu...,,[],,


In [25]:
normalized_json.columns

Index(['entities', 'journalVolume', 'journalPages', 'pmid', 'fieldsOfStudy',
       'year', 'outCitations', 's2Url', 's2PdfUrl', 'id', 'authors',
       'journalName', 'paperAbstract', 'inCitations', 'pdfUrls', 'title',
       'doi', 'sources', 'doiUrl', 'venue'],
      dtype='object')

### Flattening the JSON with jq

In [ ]:
https://neo4j.com/blog/bulk-data-import-neo4j-3-0/

In [26]:
!jq -r '.' semantic-scholar-raw/s2-corpus-000-slim

{
  "entities": [],
  "journalVolume": "",
  "journalPages": "",
  "pmid": "",
  "fieldsOfStudy": [
    "Geology"
  ],
  "year": 1984,
  "outCitations": [],
  "s2Url": "https://semanticscholar.org/paper/2fc88e5e6980652109b40f15974891b55f2b9b53",
  "s2PdfUrl": "",
  "id": "2fc88e5e6980652109b40f15974891b55f2b9b53",
  "authors": [
    {
      "name": "David K. Elliott",
      "ids": [
        "50369971"
      ]
    }
  ],
  "journalName": "",
  "paperAbstract": "",
  "inCitations": [],
  "pdfUrls": [],
  "title": "A new subfamily of the Pteraspididae (Agnatha, Heterostraci) from the upper Silurian and lower Devonian of Arctic Canada",
  "doi": "",
  "sources": [],
  "doiUrl": "",
  "venue": ""
}
{
  "entities": [],
  "journalVolume": "66 1",
  "journalPages": "\n          241-3\n        ",
  "pmid": "13395544",
  "fieldsOfStudy": [
    "Medicine",
    "Biology"
  ],
  "year": 1957,
  "outCitations": [],
  "s2Url": "https://semanticscholar.org/paper/9a7cc56f5190f56bbb13fac5134a7d44db754e9